### Pipeline test ###

In this notebook we will try a possible textual claim generation to be implemented later as part of the package

In [3]:
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize,compose
from src.evidence import FeverousRetrieverRandom
from src.pipeline import ClaimGeneratorPipeline
from src.claim import FeverousGenerator
from src.claim import ToTToGenerator
from src.claim import TextualClaim
import numpy as np
import tensorflow_text
import json
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
GlobalHydra.instance().clear()
initialize(config_path='../src/config/', job_name="test_app")

hydra.initialize()

In [4]:
cfg = compose(config_name="config_pipeline")
rng = np.random.default_rng(cfg.seed)
retrievers = [FeverousRetrieverRandom(p_dataset='../datasets/filtereddb_st_2.db',
                                    num_evidence=1,
                                    table_per_page=cfg.table_per_page,
                                    evidence_per_table=cfg.evidence_per_table,
                                    column_per_table=cfg.column_per_table,
                                    seed=23,
                                    key_strategy=strat
                                    ) for strat in ['entity','random']]


generator1 = FeverousGenerator(encoding='compact', model_path='../models/t5_train_feverous.bin')
#generator2 = ToTToGenerator(encoding='totto', model_path='../models/exported_totto_large/1648208035')
generator3 = ToTToGenerator(encoding='compact', model_path='../models/exported_totto_large/1648208035')

generators = [generator3]

pipeline = ClaimGeneratorPipeline([retrievers,generators])
claims = pipeline.generate() # Right now, FeverousRetriever doesn't support an input table

json_evidence = TextualClaim.to_json(claims)

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(json_evidence, f, ensure_ascii=False, indent=4)


2022-05-17 09:27:00.315708: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 131596288 exceeds 10% of free system memory.
/home/akatief/PycharmProjects/eurecom-evidence-generator/venv/lib/python3.9/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
